# UTS 
Nama : R.Bella Aprilia Damayanti <br>
NIM : 200411100082 <br>
Kelas : PSD B

## Klasifikasi merupakan kata serapan dari bahasa Belanda, classificatie, yang sendirinya berasal dari bahasa Prancis classification.


## Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
%cd /content/drive/My Drive/prosaindata/

In [ ]:
#install library
!pip install sastrawi
!pip install swifter
!pip install gensim

In [ ]:
#import library
import pandas as pd
import numpy as np
from string import punctuation
import re
import nltk

In [ ]:
#load data
df = pd.read_excel("tugas/dataset/Data_TA.xlsx")
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df['Label'].value_counts()

## Preprocessing

### 1. Symbol & Punctuation Removal, case folding

In [ ]:
#proses menghilangkan simbol dan emoji
def remove_text_special (text):
  text = text.replace('\\t',"").replace('\\n',"").replace('\\u',"").replace('\\',"")
  text = text.encode('ascii', 'replace').decode('ascii')
  return text.replace("http://"," ").replace("https://", " ")
df['Abstrak'] = df['Abstrak'].apply(remove_text_special)
print(df['Abstrak'])

In [ ]:
def remove_tanda_baca(text):
  text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
  return text

df['Abstrak'] = df['Abstrak'].apply(remove_tanda_baca)
df['Abstrak'].head(20)

In [ ]:
#proses menghilangkan angka
def remove_numbers (text):
  return re.sub(r"\d+", "", text)
df['Abstrak'] = df['Abstrak'].apply(remove_numbers)
df['Abstrak']

In [ ]:
#proses casefolding
def casefolding(Comment):
  Comment = Comment.lower()
  return Comment
df['Abstrak'] = df['Abstrak'].apply(casefolding)
df['Abstrak']

### 2. Word Normalization

In [ ]:
#proses tokenisasi
# from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
# def word_tokenize(text):
#   tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#   return tokenizer.tokenize(text)

df['review_token'] = df['Abstrak'].apply(lambda sentence: nltk.word_tokenize(sentence))
df['review_token']

In [ ]:
#Normalisasi kata tidak baku
normalize = pd.read_excel("tugas/dataset/Normalization Data.xlsx")

normalize_word_dict = {}

for row in normalize.iterrows():
  if row[0] not in normalize_word_dict:
    normalize_word_dict[row[0]] = row[1]

def normalized_term(comment):
  return [normalize_word_dict[term] if term in normalize_word_dict else term for term in comment]

df['comment_normalize'] = df['review_token'].apply(normalized_term)
df['comment_normalize'].head(20)

### 3. Tokenizing

### 4. Stopwords Removal

In [ ]:
#Stopword Removal
nltk.download('stopwords')
from nltk.corpus import stopwords
txt_stopwords = stopwords.words('indonesian')

def stopwords_removal(filtering) :
  filtering = [word for word in filtering if word not in txt_stopwords]
  return filtering

df['stopwords_removal'] = df['comment_normalize'].apply(stopwords_removal)
df['stopwords_removal'].head(20)

In [ ]:
#stopword removal 2
data_stopwords = pd.read_excel("tugas/dataset/list_stopwords.xlsx")
print(data_stopwords)

def stopwords_removal2(filter) :
  filter = [word for word in filter if word not in data_stopwords]
  return filter

df['stopwords_removal_final'] = df['stopwords_removal'].apply(stopwords_removal2)
df['stopwords_removal_final'].head(20)

### 5. Stemming

In [ ]:
#proses stem
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import swifter
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming (term):
  return stemmer.stem(term)

term_dict = {}
for document in df['stopwords_removal_final']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ''


In [ ]:
print(len(term_dict))
print("-----------------------------")

In [ ]:
for term in term_dict:
  term_dict[term] = stemming(term)
  print(term,":",term_dict[term])

print(term_dict)
print("-----------------------------")

In [ ]:
def get_stemming(document):
  return [term_dict[term] for term in document]

In [ ]:
df['stemming'] = df['stopwords_removal_final'].swifter.apply(get_stemming)

In [ ]:
print(df['stemming'])

In [ ]:
df.head(20)

## PCA

**Principal Component Analysis (PCA)** adalah salah satu metode reduksi dimensi pada machine learning. PCA akan memilih œvariabel-variabel yang mampu menjelaskan sebagian besar variabilitas data. PCA mengurangi dimensi dengan membentuk variabel-variabel baru yang disebut Principal Components. Principal Components yang merupakan kombinasi linier dari variabel-variabel lama. Penghitungan Varians dan Principal Component ini dapat dilakukan dengan menggunakan konsep nilai eigen (eigenvalue) dan vektor eigen (eigenvector) dari ilmu Aljabar Linier.

**Principal Component Analysis (PCA)** tentunya punya banyak manfaat dalam proses analisis data, misal:

*   Mengatasi multikolinieritas yang pada metode parametrik tertentu merupakan 
*   asumsi yang harus dipenuhi
*   Mereduksi jumlah variabel yang akan dimasukkan ke model
*   Jumlah variabel yang lebih sedikit tentu akan menyederhanakan model
*   Juga mempercepat proses komputasi

### Normalisasi Data (Tf IDF)



In [ ]:
def joinkata(data):
  kalimat = ""
  for i in data:
    kalimat += i
    kalimat += " "
  return kalimat

text = df['stemming'].swifter.apply(joinkata)
text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_separate = tfidf_vectorizer.fit_transform(text)

df_tfidf = pd.DataFrame(
    tfidf_separate.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df.index
)
X = df_tfidf.values
Y = df['Label']
df_tfidf

In [ ]:
Y

### splitting data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)
print("Jumlah Data training : ", len(X_train))
print("Jumlah Data test : ", len(X_test))

## Implementasi PCA

Melakukan skenario uji coba dengan mereduksi fitur dengan n component sebanyak 2,4,6,8,10

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components =2)

X_train_2 = pca.fit_transform(X_train)
X_test_2 = pca.transform(X_test)

pca.explained_variance_ratio_

## Implementasi metode Naive Bayes

## - Naive Bayes adalah metode klasifikasi data berdasarkan probabilitas yang bisa saja terjadi di masa depan. Metode ini dianggap simpel dan efektif untuk diterapkan dalam analisis perusahaan.

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_model = GaussianNB()
gnb_model.fit(X_train_2, Y_train)

### Menghitung akurasi

In [ ]:
from sklearn.metrics import accuracy_score
Y_pred = gnb_model.predict(X_test_2)
print (" GNB Accuracy : ",
    accuracy_score(Y_test,Y_pred)*100)

## Implementasi metode KNN


## - KNN merupakan algoritma klasifikasi yang paling sederhana dalam mengklasifikasikan sebuah gambar kedalam sebuah label. Metode ini mudah dipahami dibandingkan metode lain karena mengklasifikasikan berdasarkan jarak terdekat dengan objek lain (tetangga).

In [ ]:
# import knn
from sklearn.neighbors import KNeighborsClassifier
KNN_model = KNeighborsClassifier(n_neighbors=5)
KNN_model.fit(X_train_2, Y_train)

### Menghitung akurasi

In [ ]:
from sklearn.metrics import accuracy_score
Y_pred = KNN_model.predict(X_test_2)
print (" KNN Accuracy : ",
    accuracy_score(Y_test,Y_pred)*100)